In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import etree

In [2]:

file_name = "status_invest_vti.html"
sorted_items = 32

In [3]:
pd.set_option("display.max_rows", 1000) # AUMENTA O LIMITE DE VISUALIZAÇÃO DE LINHAS

In [4]:
df = pd.read_html(file_name)[0] 
df.columns = ["name", "category", "quantity", "participation", "market_cap"]

In [5]:
tickers = (
    df["name"]
    .str
    .extract(r"(?P<ticker>\w+) (?P<company_name>.+?)(?: arrow_forward)")
)

df = (
    df.join(tickers)
    .drop(['name'], axis=1)
)

df["participation"] = (
    df["participation"]
    .str
    .extract(r"(?P<participation>[\d.,]+)")
    .participation
    .str
    .replace(",", ".")
    .astype(float)
)

df["market_cap"] = (
    df["market_cap"]
    .str
    .extract(r"(?P<participation>[\d.,]+)")
    .participation
    .str
    .replace(".", "", regex=False)
    .str
    .replace(",", ".", regex=False)
    .astype(float)
)

In [6]:
df = df.loc[df["category"] == "STOCKS"] ## DEIXANDO APENAS AS STOCKS

df.to_excel("total_stocks.xls")

C:\Users\jonathan.santos\AppData\Local\Temp\ipykernel_24400\3321783720.py:3: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel("total_stocks.xls")


In [7]:
df = df.sample(n=sorted_items) #SELECIONA 32 ITENS

In [8]:
def clean_data(text: str) -> str:
    text = text.replace("%", "").replace("-", "").replace(",", ".")
    if len(text) == 0:
        return np.nan
    return text

In [9]:
def get_data(ticker: str) -> str:
    url = f"https://statusinvest.com.br/acoes/eua/{ticker}"
    
    dy_xpath = '//*[@id="indicators-section"]/div[2]/div/div[1]/div/div[1]/div/div/strong'
    pl_xpath =  '//*[@id="indicators-section"]/div[2]/div/div[1]/div/div[2]/div/div/strong'
    div_liq_ebitda_xpath = '//*[@id="indicators-section"]/div[2]/div/div[2]/div/div[2]/div/div/strong'
    cagr_profit_5y_xpath = '//*[@id="indicators-section"]/div[2]/div/div[5]/div/div[2]/div/div/strong'

    response = requests.get(url=url, timeout=60)
    soup = BeautifulSoup(response.content, 'html.parser')
    dom = etree.HTML(str(soup))

    dy = clean_data(dom.xpath(dy_xpath)[0].text)
    pl = clean_data(dom.xpath(pl_xpath)[0].text)
    div_liq_ebitda = clean_data(dom.xpath(div_liq_ebitda_xpath)[0].text)
    cagr_profit_5y = clean_data(dom.xpath(cagr_profit_5y_xpath)[0].text)
    return {
        "dy": dy, 
        "pl": pl,
        "div_liq_ebitda": div_liq_ebitda, 
        "cagr_profit_5y": cagr_profit_5y, 
        "url": url
        }

In [10]:
complementary_df = df.apply(lambda row: get_data(row.ticker), axis='columns', result_type='expand')
complementary_df = complementary_df.astype(
    {
        "dy": float,
        "pl": float,
        "div_liq_ebitda": float,
        "cagr_profit_5y": float,
    }
)

In [12]:

data = complementary_df.join(df, how='left')

data = data.loc[:, [
    "pl", "dy", "div_liq_ebitda", "cagr_profit_5y", "ticker",
    "company_name", "url"
    ]
]

data.to_excel("32_random_stocks_with_data.xls")

C:\Users\jonathan.santos\AppData\Local\Temp\ipykernel_24400\70262535.py:9: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  data.to_excel("40_random_stocks_with_data.xls")


In [13]:
data

,pl,dy,div_liq_ebitda,cagr_profit_5y,ticker,company_name,url
1097,8.90,5.08,9.93,12.81,UMPQ,Umpqua Holdings Corp.,https://statusinvest.com.br/acoes/eua/UMPQ
1275,21.61,NaN,NaN,60.59,ONTO,Onto Innovation Inc.,https://statusinvest.com.br/acoes/eua/ONTO
607,12.32,1.11,NaN,7.21,SBNY,Signature Bank/New York NY,https://statusinvest.com.br/acoes/eua/SBNY
2281,13.90,NaN,1.28,20.57,KE,Kimball Electronics Inc.,https://statusinvest.com.br/acoes/eua/KE
2101,1.48,NaN,NaN,NaN,FMTX,Forma Therapeutics Holdings Inc.,https://statusinvest.com.br/acoes/eua/FMTX
2109,11.55,NaN,0.93,NaN,AXGN,Axogen Inc.,https://statusinvest.com.br/acoes/eua/AXGN
1065,6.41,1.31,0.98,NaN,OLN,Olin Corp.,https://statusinvest.com.br/acoes/eua/OLN
1418,4.49,NaN,6.69,NaN,RIG,Transocean Ltd.,https://statusinvest.com.br/acoes/eua/RIG
2529,4.47,NaN,0.92,14.80,KIRK,Kirkland's Inc.,https://statusinvest.com.br/acoes/eua/KIRK
325,9.46,4.22,2.71,14.15,IP,International Paper Co.,https://statusinvest.com.br/acoes/eua/IP
